# IDPP before NEB

Starting out with LAMMPS data files, and then using ASE to get the [IDPP interpolated images](https://wiki.fysik.dtu.dk/ase/tutorials/neb/idpp.html)

In [1]:
from ase.neb import NEB
from ase import Atom, Atoms
from ase.io import lammpsdata, read 
from ase.constraints import FixBondLengths
# from gpaw.utilities.watermodel import FixBondLengthsWaterModel as FixBondLengths
import os
import numpy as np
from ase.constraints import FixAtoms
from ase.calculators.emt import EMT

# Read write only
import re
from ase.atoms import Atoms
from ase.calculators.lammps import Prism, convert
from ase.utils import reader, writer
from collections import namedtuple

# Debugging
from IPython.core.debugger import set_trace

## NamedTuple Definitions
These are used from here on out

In [2]:
BondInfo = namedtuple('BondInfo', ['bonded_to', 'bond_type', 'bonded_from'])

In [3]:
def assoc_bonds(bond_array):
    res = []
    for eind, elem in enumerate(bond_array):
        if '_' not in elem:
            lbonds = elem.split(",") # 
            for la in lbonds:
                ll = la.split('(')
                res.append(
                    BondInfo(
                        bonded_to = int(ll[0].strip(')')),
                        bond_type = int(ll[1].strip(')')),
                        bonded_from = int(eind)
                    )
                )
    return res

NamedTuple for angles 

In [4]:
AngleInfo = namedtuple('AngleInfo', ['angle_to_1', 'angle_to_2', 'angle_type', 'central_angle_atom'])

In [5]:
def assoc_angles(angle_array):
    res = []
    for eind, elem in enumerate(angle_array):
        if '_' not in elem:
            langles = elem.split(",") # angle_to_1 - angle_to_2 (type)
            for la in langles:
                l1 = la.split('-') # gives angle_to_1
                l2 = l1[1].split('(') # gives angle_to_2 and type
                res.append(
                    AngleInfo(
                        angle_to_1 = int(l1[0].strip(')')),
                        angle_to_2 = int(l2[0].strip(')')),
                        angle_type = int(l2[1].strip(')')),
                        central_angle_atom = int(eind)
                    )
                )
    return res

NamedTuple for dihedrals

In [6]:
DihedralInfo = namedtuple('DihedralInfo', ['index1', 'index2', 'index3', 'dihedral_type', 'index0'])

In [7]:
def assoc_dihedrals(dihedral_array):
    res = []
    for eind, elem in enumerate(dihedral_array):
        if '_' not in elem:
            ldihedrals = elem.split(",") # various dihedrals for the current eind
            for la in ldihedrals:
                l1 = la.split('-') # splits into indices, (the last has the type)
                l2 = l1[-1].split('(') # gives angle_to_2 and type
                res.append(
                    DihedralInfo(
                        index1 = int(l1[0].strip(')')),
                        index2 = int(l1[1].strip(')')),
                        index3 = int(l2[0].strip(')')),
                        dihedral_type = int(l2[-1].strip(')')),
                        index0 = int(eind)
                    )
                )
    return res

# Read/Write Functions

The modified function to read in the LAMMPS data file: 

In [8]:
def my_read_lammps_data(filename, Z_of_type=None, style="full",
                     sort_by_id=False, units="metal"):
    """Method which reads a LAMMPS data file.

    sort_by_id: Order the particles according to their id. Might be faster to
    switch it off.
    Units are set by default to the style=metal setting in LAMMPS.
    """
    
    fileobj = open(filename, "r")
    
    # load everything into memory
    lines = fileobj.readlines()

    # begin read_lammps_data
    comment = None
    N = None
    # N_types = None
    xlo = None
    xhi = None
    ylo = None
    yhi = None
    zlo = None
    zhi = None
    xy = None
    xz = None
    yz = None
    pos_in = {}
    travel_in = {}
    mol_id_in = {}
    charge_in = {}
    mass_in = {}
    vel_in = {}
    bonds_in = []
    angles_in = []
    dihedrals_in = []

    sections = [
        "Atoms",
        "Velocities",
        "Masses",
        "Charges",
        "Ellipsoids",
        "Lines",
        "Triangles",
        "Bodies",
        "Bonds",
        "Angles",
        "Dihedrals",
        "Impropers",
        "Impropers Pair Coeffs",
        "PairIJ Coeffs",
        "Pair Coeffs",
        "Bond Coeffs",
        "Angle Coeffs",
        "Dihedral Coeffs",
        "Improper Coeffs",
        "BondBond Coeffs",
        "BondAngle Coeffs",
        "MiddleBondTorsion Coeffs",
        "EndBondTorsion Coeffs",
        "AngleTorsion Coeffs",
        "AngleAngleTorsion Coeffs",
        "BondBond13 Coeffs",
        "AngleAngle Coeffs",
    ]
    header_fields = [
        "atoms",
        "bonds",
        "angles",
        "dihedrals",
        "impropers",
        "atom types",
        "bond types",
        "angle types",
        "dihedral types",
        "improper types",
        "extra bond per atom",
        "extra angle per atom",
        "extra dihedral per atom",
        "extra improper per atom",
        "extra special per atom",
        "ellipsoids",
        "lines",
        "triangles",
        "bodies",
        "xlo xhi",
        "ylo yhi",
        "zlo zhi",
        "xy xz yz",
    ]
    sections_re = "(" + "|".join(sections).replace(" ", "\\s+") + ")"
    header_fields_re = "(" + "|".join(header_fields).replace(" ", "\\s+") + ")"

    section = None
    header = True
    for line in lines:
        if comment is None:
            comment = line.rstrip()
        else:
            line = re.sub("#.*", "", line).rstrip().lstrip()
            if re.match("^\\s*$", line):  # skip blank lines
                continue

        # check for known section names
        m = re.match(sections_re, line)
        if m is not None:
            section = m.group(0).rstrip().lstrip()
            header = False
            continue

        if header:
            field = None
            val = None
            # m = re.match(header_fields_re+"\s+=\s*(.*)", line)
            # if m is not None: # got a header line
            #   field=m.group(1).lstrip().rstrip()
            #   val=m.group(2).lstrip().rstrip()
            # else: # try other format
            #   m = re.match("(.*)\s+"+header_fields_re, line)
            #   if m is not None:
            #       field = m.group(2).lstrip().rstrip()
            #       val = m.group(1).lstrip().rstrip()
            m = re.match("(.*)\\s+" + header_fields_re, line)
            if m is not None:
                field = m.group(2).lstrip().rstrip()
                val = m.group(1).lstrip().rstrip()
            if field is not None and val is not None:
                if field == "atoms":
                    N = int(val)
                # elif field == "atom types":
                #     N_types = int(val)
                elif field == "xlo xhi":
                    (xlo, xhi) = [float(x) for x in val.split()]
                elif field == "ylo yhi":
                    (ylo, yhi) = [float(x) for x in val.split()]
                elif field == "zlo zhi":
                    (zlo, zhi) = [float(x) for x in val.split()]
                elif field == "xy xz yz":
                    (xy, xz, yz) = [float(x) for x in val.split()]

        if section is not None:
            fields = line.split()
            if section == "Atoms":  # id *
                id = int(fields[0])
                if style == "full" and (len(fields) == 7 or len(fields) == 10):
                    # id mol-id type q x y z [tx ty tz]
                    pos_in[id] = (
                        int(fields[2]),
                        float(fields[4]),
                        float(fields[5]),
                        float(fields[6]),
                    )
                    mol_id_in[id] = int(fields[1])
                    charge_in[id] = float(fields[3])
                    if len(fields) == 10:
                        travel_in[id] = (
                            int(fields[7]),
                            int(fields[8]),
                            int(fields[9]),
                        )
                elif style == "atomic" and (
                        len(fields) == 5 or len(fields) == 8
                ):
                    # id type x y z [tx ty tz]
                    pos_in[id] = (
                        int(fields[1]),
                        float(fields[2]),
                        float(fields[3]),
                        float(fields[4]),
                    )
                    if len(fields) == 8:
                        travel_in[id] = (
                            int(fields[5]),
                            int(fields[6]),
                            int(fields[7]),
                        )
                elif (style in ("angle", "bond", "molecular")
                      ) and (len(fields) == 6 or len(fields) == 9):
                    # id mol-id type x y z [tx ty tz]
                    pos_in[id] = (
                        int(fields[2]),
                        float(fields[3]),
                        float(fields[4]),
                        float(fields[5]),
                    )
                    mol_id_in[id] = int(fields[1])
                    if len(fields) == 9:
                        travel_in[id] = (
                            int(fields[6]),
                            int(fields[7]),
                            int(fields[8]),
                        )
                elif (style == "charge"
                      and (len(fields) == 6 or len(fields) == 9)):
                    # id type q x y z [tx ty tz]
                    pos_in[id] = (
                        int(fields[1]),
                        float(fields[3]),
                        float(fields[4]),
                        float(fields[5]),
                    )
                    charge_in[id] = float(fields[2])
                    if len(fields) == 9:
                        travel_in[id] = (
                            int(fields[6]),
                            int(fields[7]),
                            int(fields[8]),
                        )
                else:
                    raise RuntimeError(
                        "Style '{}' not supported or invalid "
                        "number of fields {}"
                        "".format(style, len(fields))
                    )
            elif section == "Velocities":  # id vx vy vz
                vel_in[int(fields[0])] = (
                    float(fields[1]),
                    float(fields[2]),
                    float(fields[3]),
                )
            elif section == "Masses":
                mass_in[int(fields[0])] = float(fields[1])
            elif section == "Bonds":  # id type atom1 atom2
                bonds_in.append(
                    (int(fields[1]), int(fields[2]), int(fields[3]))
                )
            elif section == "Angles":  # id type atom1 atom2 atom3
                angles_in.append(
                    (
                        int(fields[1]),
                        int(fields[2]),
                        int(fields[3]),
                        int(fields[4]),
                    )
                )
            elif section == "Dihedrals":  # id type atom1 atom2 atom3 atom4
                dihedrals_in.append(
                    (
                        int(fields[1]),
                        int(fields[2]),
                        int(fields[3]),
                        int(fields[4]),
                        int(fields[5]),
                    )
                )

    # set cell
    cell = np.zeros((3, 3))
    cell[0, 0] = xhi - xlo
    cell[1, 1] = yhi - ylo
    cell[2, 2] = zhi - zlo
    if xy is not None:
        cell[1, 0] = xy
    if xz is not None:
        cell[2, 0] = xz
    if yz is not None:
        cell[2, 1] = yz

    # initialize arrays for per-atom quantities
    positions = np.zeros((N, 3))
    numbers = np.zeros((N), int)
    ids = np.zeros((N), int)
    types = np.zeros((N), int)
    if len(vel_in) > 0:
        velocities = np.zeros((N, 3))
    else:
        velocities = None
    if len(mass_in) > 0:
        masses = np.zeros((N))
    else:
        masses = None
    if len(mol_id_in) > 0:
        mol_id = np.zeros((N), int)
    else:
        mol_id = None
    if len(charge_in) > 0:
        charge = np.zeros((N), float)
    else:
        charge = None
    if len(travel_in) > 0:
        travel = np.zeros((N, 3), int)
    else:
        travel = None
    if len(bonds_in) > 0:
        bonds = [""] * N
    else:
        bonds = None
    if len(angles_in) > 0:
        angles = [""] * N
    else:
        angles = None
    if len(dihedrals_in) > 0:
        dihedrals = [""] * N
    else:
        dihedrals = None

    ind_of_id = {}
    # copy per-atom quantities from read-in values
    for (i, id) in enumerate(pos_in.keys()):
        # by id
        ind_of_id[id] = i
        if sort_by_id:
            ind = id - 1
        else:
            ind = i
        type = pos_in[id][0]
        positions[ind, :] = [pos_in[id][1], pos_in[id][2], pos_in[id][3]]
        if velocities is not None:
            velocities[ind, :] = [vel_in[id][0], vel_in[id][1], vel_in[id][2]]
        if travel is not None:
            travel[ind] = travel_in[id]
        if mol_id is not None:
            mol_id[ind] = mol_id_in[id]
        if charge is not None:
            charge[ind] = charge_in[id]
        ids[ind] = id
        # by type
        types[ind] = type
        if Z_of_type is None:
            numbers[ind] = type
        else:
            numbers[ind] = Z_of_type[type]
        if masses is not None:
            masses[ind] = mass_in[type]
    # convert units
    positions = convert(positions, "distance", units, "ASE")
    cell = convert(cell, "distance", units, "ASE")
    if masses is not None:
        masses = convert(masses, "mass", units, "ASE")
    if velocities is not None:
        velocities = convert(velocities, "velocity", units, "ASE")

    # create ase.Atoms
    at = Atoms(
        positions=positions,
        numbers=numbers,
        masses=masses,
        cell=cell,
        pbc=[True, True, True],
    )
    # set velocities (can't do it via constructor)
    if velocities is not None:
        at.set_velocities(velocities)
    at.arrays["id"] = ids
    at.arrays["type"] = types
    if travel is not None:
        at.arrays["travel"] = travel
    if mol_id is not None:
        at.arrays["mol-id"] = mol_id
    if charge is not None:
        at.arrays["initial_charges"] = charge
        at.arrays["mmcharges"] = charge.copy()

    if bonds is not None:
        for (type, a1, a2) in bonds_in:
            i_a1 = ind_of_id[a1]
            i_a2 = ind_of_id[a2]
            if len(bonds[i_a1]) > 0:
                bonds[i_a1] += ","
            bonds[i_a1] += "%d(%d)" % (i_a2, type)
        for i in range(len(bonds)):
            if len(bonds[i]) == 0:
                bonds[i] = "_"
        at.arrays["bonds"] = np.array(bonds)

    if angles is not None:
        for (type, a1, a2, a3) in angles_in:
            i_a1 = ind_of_id[a1]
            i_a2 = ind_of_id[a2]
            i_a3 = ind_of_id[a3]
            if len(angles[i_a2]) > 0:
                angles[i_a2] += ","
            angles[i_a2] += "%d-%d(%d)" % (i_a1, i_a3, type)
        for i in range(len(angles)):
            if len(angles[i]) == 0:
                angles[i] = "_"
        at.arrays["angles"] = np.array(angles)

    if dihedrals is not None:
        for (type, a1, a2, a3, a4) in dihedrals_in:
            i_a1 = ind_of_id[a1]
            i_a2 = ind_of_id[a2]
            i_a3 = ind_of_id[a3]
            i_a4 = ind_of_id[a4]
            if len(dihedrals[i_a1]) > 0:
                dihedrals[i_a1] += ","
            dihedrals[i_a1] += "%d-%d-%d(%d)" % (i_a2, i_a3, i_a4, type)
        for i in range(len(dihedrals)):
            if len(dihedrals[i]) == 0:
                dihedrals[i] = "_"
        at.arrays["dihedrals"] = np.array(dihedrals)

    at.info["comment"] = comment
    
    fileobj.close()

    return at


The modified function to write out LAMMPS data files: 

In [9]:
def my_write_lammps_data(filename, atoms, specorder=None, force_skew=False,
                      prismobj=None, velocities=False, units="metal",
                      atom_style='atomic'):
    """Write atomic structure data to a LAMMPS data file."""

    # FIXME: We should add a check here that the encoding of the file object
    #        is actually ascii once the 'encoding' attribute of IOFormat objects
    #        starts functioning in implementation (currently it doesn't do
    #         anything).
    
    fd = open(filename, 'w') # Open file to write
    
    if isinstance(atoms, list):
        if len(atoms) > 1:
            raise ValueError(
                "Can only write one configuration to a lammps data file!"
            )
        atoms = atoms[0]

    if hasattr(fd, "name"):
        fd.write("{0} (written by ASE) \n\n".format(fd.name))
    else:
        fd.write("(written by ASE) \n\n")
    
    symbols = atoms.get_chemical_symbols()
    n_atoms = len(symbols)
    fd.write("{0} \t atoms \n".format(n_atoms))
    
    # Get an array for the LAMMPS IDs
    if "id" in atoms.arrays.keys():
        lammps_id = np.array(atoms.arrays["id"])
    else:
        lammps_id = np.arange(1, n_atoms) # LAMMPS IDs start from 1 

    if specorder is None:
        # This way it is assured that LAMMPS atom types are always
        # assigned predictably according to the alphabetic order
        species = sorted(set(symbols))
    else:
        # To index elements in the LAMMPS data file
        # (indices must correspond to order in the potential file)
        species = specorder
    n_atom_types = len(species)
    fd.write("{0}  atom types\n".format(n_atom_types))

    # Bonds and bond types
    if "bonds" in atoms.arrays.keys():
        bond_flag = True
        t_bonds = assoc_bonds(atoms.arrays["bonds"])
        n_bonds = len(t_bonds) # Number of bonds
        # Write out the bonds
        fd.write("{0} \t bonds \n".format(n_bonds))
        # Get the bond types
        bond_types = np.array( [i.bond_type for i in t_bonds] ) 
        n_bond_types = len(np.unique(bond_types))
        # Write out the bond type numbers
        fd.write("{0}  bond types\n".format(n_bond_types))
        
    # Angles and angle types
    if "angles" in atoms.arrays.keys():
        angle_flag = True
        t_angles = assoc_angles(atoms.arrays["angles"])
        n_angles = len(t_angles) # Number of angles
        # Write out the angles
        fd.write("{0} \t angles \n".format(n_angles))
        # Get the angle types
        angle_types = np.array( [i.angle_type for i in t_angles] ) 
        n_angle_types = len(np.unique(angle_types))
        # Write out the dihedral type numbers
        fd.write("{0}  angle types\n".format(n_angle_types))
        
    # Dihedrals and dihedral types
    if "dihedrals" in atoms.arrays.keys():
        dihedral_flag = True
        t_dihedrals = assoc_dihedrals(atoms.arrays["dihedrals"])
        n_dihedrals = len(t_dihedrals) # Number of angles
        # Write out the dihedrals
        fd.write("{0} \t dihedrals \n".format(n_dihedrals))
        # Get the dihedral types
        dihedral_types = np.array( [i.dihedral_type for i in t_dihedrals] ) 
        n_dihedral_types = len(np.unique(dihedral_types))
        # Write out the dihedral type numbers
        fd.write("{0}  dihedral types\n".format(n_dihedral_types))
    else:
        dihedral_flag = False
    
    if prismobj is None:
        p = Prism(atoms.get_cell())
    else:
        p = prismobj

    # Get cell parameters and convert from ASE units to LAMMPS units
    xhi, yhi, zhi, xy, xz, yz = convert(p.get_lammps_prism(), "distance",
                                        "ASE", units)
    
    fd.write("\n") # Blank line before box limits
    
    fd.write("0.0 {0:23.17g}  xlo xhi\n".format(xhi))
    fd.write("0.0 {0:23.17g}  ylo yhi\n".format(yhi))
    fd.write("0.0 {0:23.17g}  zlo zhi\n".format(zhi))

    if force_skew or p.is_skewed():
        fd.write(
            "{0:23.17g} {1:23.17g} {2:23.17g}  xy xz yz\n".format(
                xy, xz, yz
            )
        )
    fd.write("\n") # Blank line before Atoms

    # Write (unwrapped) atomic positions.  If wrapping of atoms back into the
    # cell along periodic directions is desired, this should be done manually
    # on the Atoms object itself beforehand.
    fd.write("Atoms \n\n")
    pos = p.vector_to_lammps(atoms.get_positions(), wrap=False)

    if atom_style == 'atomic':
        for i, (lmp_id, r) in enumerate(zip(lammps_id, pos)):
            # Check for LAMMPS atom type
            if "type" in atoms.arrays.keys():
                s = int( atoms.arrays["type"][i] )    
            else:
                s = species.index(symbols[i]) + 1
            # Convert position from ASE units to LAMMPS units
            r = convert(r, "distance", "ASE", units)
            fd.write(
                "{0:>6} {1:>3} {2:23.17g} {3:23.17g} {4:23.17g}\n".format(
                    *(lmp_id, s) + tuple(r)
                )
            )
    elif atom_style == 'charge':
        charges = atoms.get_initial_charges()
        for i, (lmp_i, q, r) in enumerate(zip(lammps_id, charges, pos)):
            # Check for LAMMPS atom type
            if "type" in atoms.arrays.keys():
                s = int( atoms.arrays["type"][i] )    
            else:
                s = species.index(symbols[i]) + 1
            # Convert position and charge from ASE units to LAMMPS units
            r = convert(r, "distance", "ASE", units)
            q = convert(q, "charge", "ASE", units)
            fd.write("{0:>6} {1:>3} {2:>5} {3:23.17g} {4:23.17g} {5:23.17g}\n"
                     .format(*(lmp_id, s, q) + tuple(r)))
    elif atom_style == 'full':
        charges = atoms.get_initial_charges()
        # The label 'mol-id' has apparenlty been introduced in read earlier,
        # but so far not implemented here. Wouldn't a 'underscored' label
        # be better, i.e. 'mol_id' or 'molecule_id'?
        if atoms.has('mol-id'):
            molecules = atoms.get_array('mol-id')
            if not np.issubdtype(molecules.dtype, np.integer):
                raise TypeError((
                    "If 'atoms' object has 'mol-id' array, then"
                    " mol-id dtype must be subtype of np.integer, and"
                    " not {:s}.").format(str(molecules.dtype)))
            if (len(molecules) != len(atoms)) or (molecules.ndim != 1):
                raise TypeError((
                    "If 'atoms' object has 'mol-id' array, then"
                    " each atom must have exactly one mol-id."))
        elif "mol-id" in atoms.arrays.keys():
            molecules = np.array(atoms.arrays["mol-id"])
        else:
            # Assigning each atom to a distinct molecule id would seem
            # preferableabove assigning all atoms to a single molecule id per
            # default, as done within ase <= v 3.19.1. I.e.,
            # molecules = np.arange(start=1, stop=len(atoms)+1, step=1, dtype=int)
            # However, according to LAMMPS default behavior,
            molecules = np.zeros(len(atoms), dtype=int)
            # which is what happens if one creates new atoms within LAMMPS
            # without explicitly taking care of the molecule id.
            # Quote from docs at https://lammps.sandia.gov/doc/read_data.html:
            #    The molecule ID is a 2nd identifier attached to an atom.
            #    Normally, it is a number from 1 to N, identifying which
            #    molecule the atom belongs to. It can be 0 if it is a
            #    non-bonded atom or if you don't care to keep track of molecule
            #    assignments.

        for i, (lmp_id, m, q, r) in enumerate(zip(lammps_id, molecules, charges, pos)):
            # Check for LAMMPS atom type
            if "type" in atoms.arrays.keys():
                s = int( atoms.arrays["type"][i] )    
            else:
                s = species.index(symbols[i]) + 1
            # Convert position and charge from ASE units to LAMMPS units
            r = convert(r, "distance", "ASE", units)
            q = convert(q, "charge", "ASE", units)
            fd.write("{0:>6} {1:>3} {2:>3} {3:>5} {4:23.17g} {5:23.17g} "
                     "{6:23.17g}\n".format(*(lmp_id, m, s, q) + tuple(r)))
    else:
        raise NotImplementedError

    if velocities and atoms.get_velocities() is not None:
        fd.write("\n\nVelocities \n\n")
        vel = p.vector_to_lammps(atoms.get_velocities())
        for i, v in enumerate(vel):
            # Convert velocity from ASE units to LAMMPS units
            v = convert(v, "velocity", "ASE", units)
            fd.write(
                "{0:>6} {1:23.17g} {2:23.17g} {3:23.17g}\n".format(
                    *(i + 1,) + tuple(v)
                )
            )
    
    # Write out the bonds
    if bond_flag:
        fd.write("\nBonds \n\n")
        # Get all the bonds 
        for i,ibond in enumerate(t_bonds):
            lmp_id_1 = lammps_id[ibond.bonded_from]
            lmp_id_2 = lammps_id[ibond.bonded_to]
            fd.write(
                "{0:>6} {1:>3} {2:>3} {3:>3}\n".format(
                    i + 1, ibond.bond_type, lmp_id_1, lmp_id_2)
            )
            
    # Write out the angles
    if angle_flag:
        fd.write("\nAngles \n\n")
        # Get all the angles 
        for i,iangle in enumerate(t_angles):
            lmp_id_1 = lammps_id[iangle.angle_to_1]
            lmp_id_2 = lammps_id[iangle.central_angle_atom]
            lmp_id_3 = lammps_id[iangle.angle_to_2]
            fd.write(
                "{0:>6} {1:>3} {2:>3} {3:>3} {4:>3}\n".format(
                    i + 1, iangle.angle_type, lmp_id_1, lmp_id_2, lmp_id_3)
            )
    
    # Write out the dihedrals
    if dihedral_flag:
        fd.write("\nDihedrals \n\n")
        # Get all the dihedrals 
        for i,idihedral in enumerate(t_dihedrals):
            lmp_id_1 = lammps_id[idihedral.index0]
            lmp_id_2 = lammps_id[idihedral.index1]
            lmp_id_3 = lammps_id[idihedral.index2]
            lmp_id_4 = lammps_id[idihedral.index3]
            fd.write(
                "{0:>6} {1:>3} {2:>3} {3:>3} {4:>3} {5:>3}\n".format(
                    i + 1, idihedral.dihedral_type, lmp_id_1, lmp_id_2, lmp_id_3, lmp_id_4)
            )
    
    
    fd.flush()
    fd.close()


Writes out all the atom coordinates in a custom format. The first non-blank, non-comment line should list N = the number of lines to follow. The N successive lines contain the following information:
ID1 x1 y1 z1
ID2 x2 y2 z2
...
IDN xN yN zN

In [10]:
def write_neb_lammps(filename, atoms, units="real"):
    """Write atom coordinates in a custom format for NEB in LAMMPS"""
    
    fd = open(filename, 'w') # Open file to write
    
    if isinstance(atoms, list):
        if len(atoms) > 1:
            raise ValueError(
                "Can only write one configuration to a lammps data file!"
            )
        atoms = atoms[0]
    
    # Comment lines 
    fd.write("# Coordinates of image in NEB, written by ASE \n")
    
    symbols = atoms.get_chemical_symbols()
    n_atoms = len(symbols)
    fd.write("{0}\n".format(n_atoms)) # Number of atoms written out
    
    # Get an array for the LAMMPS IDs
    if "id" in atoms.arrays.keys():
        lammps_id = np.array(atoms.arrays["id"])
    else:
        lammps_id = np.arange(1, n_atoms) # LAMMPS IDs start from 1 

    # Write (unwrapped) atomic positions.  If wrapping of atoms back into the
    # cell along periodic directions is desired, this should be done manually
    # on the Atoms object itself beforehand.
    p = Prism(atoms.get_cell())
    pos = p.vector_to_lammps(atoms.get_positions(), wrap=False)
    
    for i, (lmp_id, r) in enumerate(zip(lammps_id, pos)):
        r = convert(r, "distance", "ASE", units)
       # set_trace()
        fd.write(f"{lmp_id:>6} {r[0]:23.17g} {r[1]:23.17g} {r[2]:23.17g}\n")
        # fd.write(
        #         "{0:>6} {1:23.17g} {2:23.17g} {3:23.17g}\n".format(
        #             *([lmp_id],) + tuple(r)
        #         )
        #     )
    
    
    fd.flush()
    fd.close()

# Actually run stuff

In [11]:
## Variables
initFile = "initFinalData/image_0.data"
finalFile = "initFinalData/image_63.data"
n_intermediate_images = 62 # total number of images = 64
fixBonds = True # If this is false, we will just fix water atom positions
runIDPP = True # False if you want linear interpolation
# If both fixBonds and runIDPP are false, then the water molecule atoms will be fixed and linear interpolation will be performed.

In [12]:
# Functions
# for constraints
# function to loop over bonds 
# For water molecules: skip index idx
def rigid(atoms, idx):
    rattle = ([(3 * i + j + idx, 3 * i + (j + 1) % 3 + idx)
                        for i in range(len(atoms[idx:]) // 3)
                        for j in [0, 1, 2]])
    rattle = FixBondLengths(rattle)
    return rattle

# Make the given THF molecule rigid, starting from idx
def rigidTHF(atoms, idx):
    ringBonds = ([(idx, idx+1), 
              (idx+1, idx+2),
              (idx+2, idx+3),
              (idx+3, idx+4),
              (idx+4, idx)])
    ringBonds = FixBondLengths(ringBonds)
    return ringBonds

# Fix atoms whose symbols are given in a list
def fixWat(atoms, symbols=['H', 'He']):
    fixAtomConstraint = FixAtoms(indices=[atom.index for atom in atoms if atom.symbol == symbols[0] or atom.symbol == symbols[1]])
    return fixAtomConstraint

Read in the initial and final states from a LAMMPS data file: 

In [13]:
initial = my_read_lammps_data(initFile, Z_of_type=None, style='full', sort_by_id=False, units='real')
# t_dihedrals = assoc_dihedrals(initial.arrays["dihedrals"])
final = my_read_lammps_data(finalFile, Z_of_type=None, style='full', sort_by_id=False, units='real')

Generate blank images:

In [14]:
images = [initial]

# We want intermediate images 
for i in range(n_intermediate_images):
    images.append(initial.copy())

# Add the final image
images.append(final)

if fixBonds:
    # Rigid bond constraint
    for image in images:
        numAtoms = image.positions.shape[0]
        fixBondWat = rigid(image,1) 
        image.set_constraint(fixBondWat)
else:
    # Fix water molecules constraint 
    for image in images:
        fixAtomC = fixWat(image, ['H', 'He']) # Constrain the water molecules
        image.set_constraint(fixAtomC)

Now run the IDPP interpolation

In [15]:
neb = NEB(images) # create the NEB 'band'

if runIDPP:
    neb.interpolate(apply_constraint=True, method='idpp') # Linear interpolation of the images
    # neb.idpp_interpolate(images, fmax=0.001, mic=True, steps=10000) # IDPP interpolation of the images
else:
    neb.interpolate(apply_constraint=True) # Linear interpolation of the images

We wish to write out the images to LAMMPS data files. Let's create a folder and write the images out to this directory.

In [16]:
if not os.path.exists('imageData'):
    os.makedirs("imageData") # Create the output directory of the images data files 

if not os.path.exists("nebInput"):
    os.makedirs("nebInput") # Create the output directory for NEB input (not needed for initial image)
    
# We will also write out the initial and final images to check that they are correct.
for i,image in enumerate(images):
    filename = "imageData/image_"+str(i)+".data"
    my_write_lammps_data(filename, image, force_skew=False, units='real', atom_style='full')
    # Write out the NEB format input files
    # here, i is the partition number 
    nebFileName = "nebInput/image_"+str(i)
    # Write out the file
    write_neb_lammps(nebFileName, image, units="real")